In [ ]:
import os
import json
import time
import numpy as np
from z3 import *
from itertools import combinations
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

# Helper functions
def parseInstance(filepath):
    with open(filepath, 'r') as file:
        lines = file.readlines()
        
        # Read m and n
        m = int(lines[0].strip())
        n = int(lines[1].strip())
        
        # Read l1 to lm
        l_values = list(map(int, lines[2].strip().split()))
        
        # Read s1 to sn
        s_values = list(map(int, lines[3].strip().split()))
        
        # Read the distance matrix
        distance_matrix = []
        for line in lines[4:]:
            row = list(map(int, line.strip().split()))
            distance_matrix.append(row)
        
        return {'m': m, 'n': n, 'l_values': l_values, 's_values': s_values, 'distance_matrix': distance_matrix}

def computeBounds(D, m, n):
    # Lower Bound: Minimum distance from the origin to the farthest point
    LB = max([D[n][i]+D[i][n] for i in range(n)])

    # Upper Bound: Sum of all distances (a very loose upper bound)
    UB = sum([sum(row) for row in D])

    return LB, UB

In [1]:
# SMT Solver Helper Functions
counter = 0
def generate_unique_bool_name():
    global counter
    counter += 1
    return f"unique_bool_{counter}"

def at_most_one_T(bools):
    if len(bools) <= 4:  # base case
        return And([Not(And(b1, b2)) for b1, b2 in combinations(bools, 2)])
    
    # recursive step
    y = Bool(generate_unique_bool_name())
    first = bools[:3]
    first.append(y)
    c_first = at_most_one_T(first)

    last = bools[3:]
    last.insert(0, Not(y))
    c_last = at_most_one_T(last)

    return And(c_first, c_last)

def exactly_one_T(bools):
    return And(at_most_one_T(bools), Or(bools))

In [ ]:
def solve_mcp(m, n, l_values, s_values, D, LB, UB, time_limit=300):
    # Decision variables
    X = [[[Bool(f'X_{k}_{i}_{j}') for j in range(n + 1)] for i in range(n + 1)] for k in range(m)]
    Y = [[Bool(f'Y_{k}_{i}') for i in range(n)] for k in range(m)]
    U = [Int(f'U_{i}') for i in range(n)]
    C = [Int(f'C_{k}') for k in range(m)]
    MaxCost = Int('MaxCost')

    # Constraints
    constraints = []

    # Domains for U
    for i in range(n):
        constraints.append(U[i] > 0)
        constraints.append(U[i] <= n)

    # C1: Each item must be delivered by exactly one courier
    for i in range(n):
        constraints.append(Or([X[k][i][j] for j in range(n + 1) if i != j for k in range(m)]))
        constraints.append(Or([X[k][j][i] for j in range(n + 1) if i != j for k in range(m)]))
        for k in range(m):
            constraints.append(at_most_one_T([X[k][j][i] for j in range(n + 1) if i != j]))

    # C2: Item assignment constraints
    for i in range(n):
        for k in range(m):
            constraints.append(Y[k][i] == Or([X[k][i][j] for j in range(n + 1) if i != j]))
            constraints.append(Y[k][i] == Or([X[k][j][i] for j in range(n + 1) if i != j]))
    for i in range(n):
        constraints.append(exactly_one_T([Y[k][i] for k in range(m)]))

    # C3: Load constraints
    for k in range(m):
        constraints.append(Sum([If(Y[k][i], s_values[i], 0) for i in range(n)]) <= l_values[k])

    # C4: Ensure couriers start and end at the origin
    for k in range(m):
        constraints.append(exactly_one_T([X[k][n][j] for j in range(n)]))
        constraints.append(exactly_one_T([X[k][i][n] for i in range(n)]))
        constraints.append(Or([Y[k][i] for i in range(n)]))

    # C5: Subtour elimination constraints
    for i in range(n):
        for j in range(n):
            if i != j:
                arc_traversed = Or([X[k][i][j] for k in range(m)])
                constraints.append(Implies(arc_traversed, U[j] > U[i]))

    # Cost constraints
    for k in range(m):
        constraints.append(C[k] == Sum([If(X[k][i][j], D[i][j], 0) for i in range(n + 1) for j in range(n + 1) if i != j]))
        constraints.append(MaxCost >= C[k])

    constraints.append(Or([MaxCost == C[k] for k in range(m)]))
    constraints.append(MaxCost <= UB)
    constraints.append(MaxCost >= LB)

    # Objective: Minimize the maximum distance traveled by any courier
    solver = Optimize()
    solver.add(constraints)
    solver.minimize(MaxCost)

    start_time = time.time()
    best_solution = None
    best_max_cost = UB

    while UB > LB and time.time() - start_time < time_limit:
        mid = (UB + LB) // 2
        solver.push()
        solver.add(MaxCost <= mid)
        solver.set('timeout', int((time_limit - (time.time() - start_time)) * 1000))  # Set timeout for remaining time
        if solver.check() == sat:
            model = solver.model()
            best_solution = {
                'max_distance': model.evaluate(MaxCost).as_long(),
                'assignments': [(k, i + 1) for k in range(m) for i in range(n) if is_true(model.evaluate(Y[k][i]))],
                'routes': [
                    [(i, j) for i in range(n + 1) for j in range(n + 1) if i != j and is_true(model.evaluate(X[k][i][j]))]
                    for k in range(m)
                ]
            }
            best_max_cost = model.evaluate(MaxCost).as_long()
            UB = best_max_cost
        else:
            LB = mid + 1
        solver.pop()

    elapsed_time = time.time() - start_time
    return best_solution, elapsed_time

In [2]:
def generate_coordinates(n):
    # Create a circular layout for the distribution points
    theta = np.linspace(0, 2 * np.pi, n, endpoint=False)
    radius = 1
    coordinates = {i: (radius * np.cos(angle), radius * np.sin(angle)) for i, angle in enumerate(theta)}
    coordinates[n] = (0, 0)  # Origin at the center
    return coordinates

def plot_routes(routes, instance, instance_name, method_name):
    n = instance['n']
    colors = ['r', 'g', 'b', 'c', 'm', 'y']

    coordinates = generate_coordinates(n)

    plt.figure(figsize=(8, 8))
    plt.grid(True)

    # Plot the nodes
    for i in range(n):
        plt.plot(*coordinates[i], 'o', color='white', markersize=10, markeredgecolor='black')
        plt.text(coordinates[i][0], coordinates[i][1], f'd{i+1}', fontsize=12, ha='right')

    # Plot the origin
    plt.plot(*coordinates[n], 'o', color='black', markersize=10)
    plt.text(coordinates[n][0], coordinates[n][1], 'origin', fontsize=12, ha='right')

    # Plot the routes
    for i, route in enumerate(routes):
        color = colors[i % len(colors)]
        for j in range(len(route) - 1):
            start = coordinates[route[j]]
            end = coordinates[route[j+1]]
            plt.arrow(start[0], start[1], end[0] - start[0], end[1] - start[1],
                      color=color, head_width=0.05, length_includes_head=True)

    plt.xlabel('x')
    plt.ylabel('y')

    # Create custom legend
    legend_elements = [Line2D([0], [0], color=colors[i % len(colors)], lw=2, label=f'Courier {i+1}')
                       for i in range(len(routes))]
    plt.legend(handles=legend_elements, loc='upper left')

    # Save plot
    if not os.path.exists('res/SMT/Optimize_Binary/route_map'):
        os.makedirs('res/SMT/Optimize_Binary/route_map')
    plt.savefig(f'res/SMT/Optimize_Binary/route_map/{instance_name}_{method_name}_route.png')
    plt.close()

def save_route_json(instance_name, method_name, time_taken, optimal, obj, sol):
    if not os.path.exists('res/SMT/Optimize_Binary/route'):
        os.makedirs('res/SMT/Optimize_Binary/route')

    route_data = {
        method_name: {
            "time": int(time_taken),
            "optimal": optimal,
            "obj": obj,
            "sol": sol
        }
    }

    json_path = f'res/SMT/Optimize_Binary/route/{instance_name}_route.json'
    if os.path.exists(json_path):
        with open(json_path, 'r') as f:
            data = json.load(f)
    else:
        data = {}

    data[method_name] = route_data[method_name]

    with open(json_path, 'w') as f:
        json.dump(data, f, indent=4)

In [3]:
def print_solution(instance_name, solution, instance, method_name, time_taken):
    if solution:
        print(f"Instance: {instance_name}")
        print(f"Max Distance: {solution['max_distance']}")
        all_routes = []
        sol = []
        for k in range(len(solution['routes'])):
            items = [i for c, i in solution['assignments'] if c == k]
            route = [instance['n']]  # Start from the origin
            current_location = instance['n']
            route_items = []
            while True:
                next_location = next((j for i, j in solution['routes'][k] if i == current_location), instance['n'])
                if next_location == instance['n']:
                    break
                route.append(next_location)
                if next_location != instance['n']:
                    route_items.append(next_location + 1)
                current_location = next_location
            route.append(instance['n'])  # End at the origin
            print(f"Courier {k + 1}: Items {route_items}")
            print(f"Courier {k + 1} Route: {route}")
            all_routes.append(route)
            sol.append(route_items)
        plot_routes(all_routes, instance, instance_name, method_name)
        save_route_json(instance_name, method_name, min(time_taken, 300), time_taken <= 300, solution['max_distance'], sol)
    else:
        print(f"Instance: {instance_name}")
        print("No solution found within the time limit")
        save_route_json(instance_name, method_name, 300, False, None, [])

In [ ]:
folder_path = 'Instances'  # Adjust this path as necessary
method_name = 'Optimize_BINARY'  # Example method name

def read_dat_files(folder_path):
    dat_files = [f for f in os.listdir(folder_path) if f.endswith('.dat')]
    instances = {}

    for dat_file in dat_files:
        file_path = os.path.join(folder_path, dat_file)
        instances[dat_file] = parseInstance(file_path)

    return instances

instances = read_dat_files(folder_path)

for instance_name, instance in instances.items():
    m = instance['m']
    n = instance['n']
    l_values = instance['l_values']
    s_values = instance['s_values']
    D = instance['distance_matrix']
    LB, UB = computeBounds(D, m, n)
    solution, time_taken = solve_mcp(m, n, l_values, s_values, D, LB, UB, time_limit=300)
    print_solution(instance_name, solution, instance, method_name, time_taken)